# LLM with Web Search and Crawl

풍부한 컨텍스틀 활용하기 위해 구글검색 된 결과의 최상위 n개의 페이지를 크롤링하여 LLM에 제공하기 위한 코드입니다.



In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True, dotenv_path="../.env") 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
AZURE_OPENAI_NANO_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_NANO_DEPLOYMENT_NAME')

## Redis Setup

This notebook uses Redis for caching web content. Make sure to have Redis running with Docker:

```bash
# Run Redis container
docker run --name redis-cache -p 6379:6379 -d redis:latest redis-server --requirepass "redis_secure_password"

# To test the connection
docker exec -it redis-cache redis-cli -a redis_secure_password ping
```

Make sure the .env file contains the correct Redis connection information.

bs4 or scrapy?

In [5]:
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
import redis
from urllib.parse import urljoin
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Redis connection parameters
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = int(os.getenv("REDIS_PORT", 6379))
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
REDIS_DB = int(os.getenv("REDIS_DB", 0))

RESULTS_COUNT = 10

# Initialize Redis connection
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    db=REDIS_DB,
    decode_responses=True  # Return strings instead of bytes
)

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    
    text = "\n".join(paragraphs)
    
    return text


async def extract_contexts_async(url_snippet_tuples):
    async def fetch_and_cache(url, snippet):
        cached_text = redis_client.get(url)
        if cached_text:
            print(f"Retrieved content from Redis cache for URL: {url}")
            return f"{cached_text}"

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        async with httpx.AsyncClient(timeout=3, follow_redirects=True) as client:
            try:
                response = await client.get(url, headers=headers)
                response.raise_for_status()
            except httpx.HTTPStatusError as e:
                if e.response.status_code == 302 and "location" in e.response.headers:
                    redirect_url = e.response.headers["location"]
                    if not redirect_url.startswith("http"):
                        redirect_url = urljoin(url, redirect_url)
                    try:
                        response = await client.get(redirect_url, headers=headers)
                        response.raise_for_status()
                    except Exception as e2:
                        print(f"Redirect request failed: {e2}")
                        return f"{snippet} "
                else:
                    print(f"Request failed: {e}")
                    return f"{snippet} "
            except httpx.HTTPError as e:
                print(f"Request failed: {e}")
                return f"{snippet} "

            selector = scrapy.Selector(text=response.text)
            paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]

            
            # Extract main text content from paragraphs
            paragraphs = [p.strip() for p in selector.css('p::text, p *::text').getall() if p.strip()]
            
            # Remove duplicate and very short paragraphs
            filtered_paragraphs = []
            seen_content = set()
            for p in paragraphs:
                # Skip very short paragraphs that are likely UI elements
                if len(p) < 5:
                    continue
                # Avoid duplicate content
                if p in seen_content:
                    continue
                seen_content.add(p)
                filtered_paragraphs.append(p)
            
            # Join the filtered paragraphs
            text = "\n".join(filtered_paragraphs)
            
            # If no paragraphs were found, try to get other text content
            if not text:
                # Try extracting text from other common content elements with UTF-8 encoding
                try:
                    # Get text content from other common content containers
                    content_texts = [t.strip() for t in selector.css('article::text, article *::text, .content::text, .content *::text, main::text, main *::text').getall() if t.strip()]         
                except Exception as e:
                    print(f"UTF-8 encoding error: {e}")
                    content_texts = []
                content_texts = [t.strip() for t in selector.css('article::text, article *::text, .content::text, .content *::text, main::text, main *::text').getall() if t.strip()]
                if content_texts:
                    text = "\n".join(content_texts)
            

            snippet_text = f"{snippet}: {text}"  
            ex = int(os.getenv("REDIS_CACHE_EXPIRED_SECOND", 604800))
            if len(text) > 200:
                redis_client.set(url, snippet_text, ex=ex)
                print(f"Stored content in Redis cache for URL: {url}")

            return snippet_text

    tasks = [asyncio.create_task(fetch_and_cache(url, snippet)) for url, snippet in url_snippet_tuples]
    return await asyncio.gather(*tasks, return_exceptions=True)

def google_search(query, num=10):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query + " -filetype:pdf",
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "filter": "1",
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해. 결과에 "삼성전자"라는 단어가 반드시 포함되도록 해.

            1. Google Search용 Query Rewrite:
            - 사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해. 
            {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())



In [8]:
from IPython.display import Markdown, display
from datetime import datetime

# inputs = [
#     "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
#     "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
#     "삼성전자 25년 제품이 작년 대비 좋아진것은",
#     "삼성전자 JBL과 하만카돈 차이점이 뭐야",
#     "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
#     "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
# ]

inputs = [
    "2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 TV 추천해줘",
    "25년 제품이 작년 대비 좋아진것은",
    "JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

inputs = ["식기세척기 중에 필터 내장형 장점이 뭐야"]


#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

for input in inputs:
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Google Search Query: {query_rewrite['google_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = google_search(query_rewrite['google_search'], RESULTS_COUNT)

    url_snippet_tuples = list(zip([results[i]["link"] for i in range(len(results))],
                                  [results[i]["snippet"] for i in range(len(results))]))
    contexts = await extract_contexts_async(url_snippet_tuples)


    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day


    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘."
    user_prompt = f"""
        너는 아래 제공하는 구글에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        컨텍스트가 부족하면 대답을 하지 말고 "모르겠다"라고 대답해.
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        구글에서 제공한 컨텍스트: {contexts}
        질문: {query_rewrite['llm_query']}
        """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))

Original Input: 식기세척기 중에 필터 내장형 장점이 뭐야
Google Search Query: 삼성전자 식기세척기 필터 내장형 장점
LLM Query: 삼성전자 식기세척기 중 필터 내장형 모델의 장점과 특징이 무엇인지 자세히 설명해 주세요. 필터 내장형이 식기세척기 성능이나 유지보수에 어떤 긍정적인 영향을 미치는지도 알려 주세요.
Stored content in Redis cache for URL: https://nosearch.com/recommendation/pick/kitchen/dish_washer/12-14%EC%9D%B8%EC%9A%A9%20%EC%82%BC%EC%84%B1
Stored content in Redis cache for URL: https://nosearch.com/product/kitchen/dish_washer/detail/DW60BB837UTES
Stored content in Redis cache for URL: https://nosearch.com/product/kitchen/dish_washer/detail/DW60BB837WTES
Stored content in Redis cache for URL: https://nosearch.com/product/kitchen/dish_washer/detail/DW60BB837WAP(%EA%B8%80%EB%9E%A8)
Stored content in Redis cache for URL: https://m.blog.naver.com/diecruel/223096395754
Stored content in Redis cache for URL: https://www.samsung.com/sec/kitchen-accessories/accessories-dw-c00rkr10-d2c/DW-C00RKR10/
Stored content in Redis cache for URL: https://electrinsight.tistory.com/entry/%EC%82%BC%EC%84%B1-%EC%

삼성전자 식기세척기 중 **필터 내장형 모델**의 장점과 특징, 그리고 필터 내장형이 식기세척기 성능과 유지보수에 미치는 긍정적인 영향에 대해 아래와 같이 상세히 설명드립니다.

---

## 삼성전자 필터 내장형 식기세척기 주요 특징과 장점

### 1. 내장형 정수 필터 탑재
- **유충, 모래, 미세먼지, 미세플라스틱** 등 물 속의 오염물질을 한번 더 걸러주는 특허 출원된 정수 필터가 내장되어 있습니다.
- 이 필터는 마지막 헹굼 과정에서 작동하며, 깨끗한 물로 세척해 위생적인 식기 세척을 가능하게 합니다.
- 필터 성능은 국제 표준시험법(ISO Coarse A4 수준)의 먼지 및 물 투과 실험을 통과하여 신뢰성이 높습니다.
- 필터는 약 12개월 또는 약 380회 사용 시 교체 알람이 울리며, 사용자가 손쉽게 교체할 수 있습니다.

### 2. 100℃ 열풍건조 및 고온 살균 기능
- 100도 이상의 고온 열풍으로 플라스틱 식기 위 남은 물방울까지 완벽히 증발시켜, 뽀송뽀송한 건조를 구현합니다.
- 젖병살균 코스 등 고온 직수와 열풍 건조로 유해세균과 바이러스(대장균, 황색포도상구균, 코로나 바이러스 유사 등)를 99.999% 살균합니다.
- 위생 관리가 중요한 가정, 특히 아기 용품 세척에 매우 안심할 수 있는 기능입니다.

### 3. AI 맞춤 세척 모드
- 세척 전 애벌세척 과정에서 식기의 오염도를 감지해 오염 정도에 따라 물 사용량, 온도, 분사 세기를 자동 조절합니다.
- AI 빅데이터 학습을 통해 사용자의 세척 패턴을 분석하여 가장 적합한 코스를 추천하고, 사용빈도에 따라 코스 순서와 옵션값을 자동으로 조정합니다.
- 이를 통해 에너지와 물 소비를 절감하며 효율적인 세척을 돕습니다.

### 4. 대화형 알림창 및 편의 기능
- 세척 과정, 관리 가이드, 옵션 설명을 문장으로 친절히 알려주는 대화형 알림창(LCD)이 탑재되어 초보자도 쉽게 사용 가능합니다.
- 상단 랙과 중간 랙의 높이 조절이 자유로워 다양한 크기와 종류의 식기를 효율적으로 수납할 수 있습니다.
- 최대 14인용 대용량 수납이 가능하며, 최대 110개 이상의 식기를 한번에 세척할 수 있습니다.

### 5. 저소음 설계 및 슬림핏 디자인
- BLDC 모터와 안전한 이중 차음재(EVA 소재)를 사용해 약 30dB 수준의 저소음을 구현하여 야간에도 조용히 사용할 수 있습니다.
- 슬림핏 디자인으로 전기레인지와 직렬 설치가 가능하여 주방 공간을 더욱 깔끔하고 일체감 있게 연출할 수 있습니다.

---

## 필터 내장형 모델이 식기세척기 성능 및 유지보수에 미치는 긍정적 영향

### 성능 측면
- **깨끗한 세척수 공급**: 필터 내장형은 수돗물 내 불순물 제거를 통해 세척 시 깨끗한 물을 사용함으로써 세척력과 위생 수준을 높입니다.
- **세균 및 바이러스 차단 강화**: 고온 살균과 열풍건조와 함께 필터가 작동하여 살균 효과를 극대화하며, 위생적인 세척 결과를 보장합니다.
- **AI 맞춤 세척과 결합된 효율성**: 오염도에 맞는 세척 최적화와 필터 정화로 에너지와 물 사용량을 줄이고 동시에 세척 품질을 유지합니다.

### 유지보수 측면
- **필터 교체 알람 기능**: 필터 교체 시기를 사용자에게 알려줘 주기적인 교체를 돕고, 필터 막힘으로 인한 성능 저하를 방지합니다.
- **셀프 교체 가능**: 필터는 사용자가 직접 쉽게 교체할 수 있어 유지보수가 간편합니다.
- **장비 내구성 향상**: 이물질로 인한 내부 손상이나 고장을 예방해 제품 수명을 연장시킵니다.
- **청결 유지 용이**: 필터 오염 방지를 위해 캡을 끝까지 돌려 조립하도록 하여, 오염 발생 가능성을 최소화합니다.

---

## 요약 및 추천

| 구분               | 내용                                                             |
|------------------|----------------------------------------------------------------|
| 필터 내장형 주요 기능  | 유충·미세먼지·플라스틱 제거, 12개월 교체 주기, 고온살균과 100℃ 열풍건조, AI 맞춤 세척 |
| 세척 성능           | 깨끗한 물로 세척 및 고온 살균, 세척 코스 자동 최적화로 세척력 향상                |
| 유지보수 편리성       | 교체 알람과 셀프 교체 가능, 내부 보호 및 수명 연장                                  |
| 사용자 편의          | 대화형 알림창, 공간 조절 가능한 3단 바구니, 저소음 운전, 슬림핏 설치                |

---

필터 내장형 삼성전자 식기세척기는 위생과 세척력, 유지보수 편의성을 동시에 갖춘 프리미엄 라인으로, 깨끗하고 안심할 수 있는 식기세척을 원하시는 고객에게 특히 추천드립니다. 815 모델 대비 837 모델 이상에서 필터 내장형 및 대화형 알림창, AI 맞춤 세척 등 고급 기능이 포함되니 예산과 필요에 따라 선택하시면 좋습니다.

---

필요하시면 특정 모델 추천이나 추가 기능 설명도 도와드리겠습니다!